In [1]:
# Step 1: Building Connection
import fxcmpy
API = ""
con = fxcmpy.fxcmpy(access_token = API,log_level = "error", server = "demo", log_file='log.txt')

# Step 2: Defining Parameters
tickers = ["EUR/USD","USD/JPY","XAU/USD"]
lot_size = 10
candles_period = "m5"
candles_number = 250

# Step 3: Creating Functions
import pandas as pd
import talib
import copy

def get_cur_sit(ticker): # Get i
    long_short = ""
    open_pos = con.get_open_positions()
    if len(open_pos) > 0:
        open_pos_cur = open_pos[open_pos["currency"] == ticker]
        if len(open_pos_cur) > 0:
            if open_pos_cur["isBuy"] == True:
                long_short = "Long"
            if open_pos_cur["isBuy"] == False:
                long_short = "Short"
    return long_short

def get_data(ticker,candles_period,candles_number):
    temp_data = con.get_candles(ticker,period = candles_period, number = candles_number)
    data = temp_data.iloc[:,[0,1,2,3,8]]
    data.columns = ["Open","Close","High","Low","Volume"]
    return data

def modify_data(data):
    data["MA20"] = talib.MA(data["Close"],timeperiod = 20)
    data["MA50"] = talib.MA(data["Close"],timeperiod = 50)
    return data

def get_signal(data,long_short):
    signal = ""
    df = copy.deepcopy(data)
    if long_short == "":
        if (df["MA20"][-1] > df["MA50"][-1]) and abs(df["MA20"][-2] - df["MA50"][-2]) < 10:
            signal = "Buy"
        if (df["MA20"][-1] < df["MA50"][-1]) and abs(df["MA20"][-2] - df["MA50"][-2]) < 10:
            signal = "Sell"
            
    elif long_short == "long":
        if abs(df["MA20"][-1] - df["MA50"][-1]) < 10:
            signal = "Close"
        elif (df["MA20"][-1] < df["MA50"][-1]) and abs(df["MA20"][-2] - df["MA50"][-2]) < 10:
            signal = "Close_Sell"
            
    elif long_short == "short":
        if abs(df["MA20"][-1] - df["MA50"][-1]) < 10:
            signal = "Close"
        elif (df["MA20"][-1] > df["MA50"][-1]) and abs(df["MA20"][-2] - df["MA50"][-2]) < 10:
            signal = "Close_Buy"
    return signal
            
def execute_order(signal,ticker):
    if signal == "Buy":
        con.open_trade(symbol = ticker, is_buy = True, is_in_pips = True, amount = lot_size, 
                        time_in_force= 'GTC', stop= -8, trailing_step = True, order_type= 'AtMarket')
    elif signal == "Sell":
        con.open_trade(symbol = ticker, is_buy = False, is_in_pips = True, amount = lot_size, 
                        time_in_force = 'GTC', stop = -8, trailing_step = True, order_type = 'AtMarket')
    elif signal == "Close":
        con.close_all_for_symbol(ticker)
    elif signal == "Close_Buy":
        con.close_all_for_symbol(ticker)
        con.open_trade(symbol = ticker, is_buy = True, is_in_pips = True, amount = lot_size, 
                        time_in_force = 'GTC', stop = -8, trailing_step = True, order_type = 'AtMarket')
    elif signal == "Close_Sell":
        con.close_all_for_symbol(ticker)
        con.open_trade(symbol = ticker, is_buy = False, is_in_pips = True, amount = lot_size, 
                        time_in_force = 'GTC', stop = -8, trailing_step = True, order_type = 'AtMarket')

# Step 4: Creating Main Loop
def main():
    try:
        for ticker in tickers:
            long_short = get_cur_sit(ticker)
            temp_data = get_data(ticker,candles_period,candles_number)
            data = modify_data(temp_data)
            signal = get_signal(data,long_short)
            execute_order(signal,ticker)
    except:
        print("There is an error")
        
# Step 5: Creating Continuous Loop
import time

start_time = time.time()
end_time = time.time() + 60*60*24

while time.time() < end_time:
    try:
        main()
        time.sleep(300-((time.time()-start_time) % 300))
    except:
        print("There is an error")
        exit()
        
# Step 6: Close Everything
for ticker in tickers:
    con.close_all_for_symbol(ticker)
con.close()

ServerError: Can not connect to FXCM Server.